In [1]:
import os
os.chdir("/Users/suncicie/Study/Project/DataMining/Competition/Tecent")# 将当前目录更换到此目录下
print os.getcwd()

/Users/suncicie/Study/Project/DataMining/Competition/Tecent


In [3]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.cross_validation import train_test_split
import time
start_time=time.time()

In [4]:
train=pd.read_csv("dataset/pre/train_user_position_ad.csv")
test=pd.read_csv("dataset/pre/test_user_position_ad.csv")

In [5]:
print train.head()
print test.head()

   Unnamed: 0  label  clickTime  creativeID   userID  positionID  \
0           0      0     170000        3089  2798058         293   
1           1      0     170000        1259   463234        6161   
2           2      0     170000        4465  1857485        7434   
3           3      0     170000        1004  2038823         977   
4           4      0     170000        1887  2015141        3688   

   connectionType  telecomsOperator  age  gender     ...       haveBaby  \
0               1                 1   25       2     ...              1   
1               1                 2   20       2     ...              0   
2               4                 1   16       1     ...              0   
3               1                 1    0       0     ...              0   
4               1                 1   35       1     ...              0   

   hometown  residence  sitesetID  positionType  adID  camgaignID  \
0         0       1301          0             3  1321          83   
1 

In [6]:
del train[train.columns[0]]
del test[test.columns[0]]
print train.columns
print test.columns

Index([u'label', u'clickTime', u'creativeID', u'userID', u'positionID',
       u'connectionType', u'telecomsOperator', u'age', u'gender', u'education',
       u'marriageStatus', u'haveBaby', u'hometown', u'residence', u'sitesetID',
       u'positionType', u'adID', u'camgaignID', u'advertiserID', u'appID',
       u'appPlatform'],
      dtype='object')
Index([u'instanceID', u'clickTime', u'creativeID', u'userID', u'positionID',
       u'connectionType', u'telecomsOperator', u'age', u'gender', u'education',
       u'marriageStatus', u'haveBaby', u'hometown', u'residence', u'sitesetID',
       u'positionType', u'adID', u'camgaignID', u'advertiserID', u'appID',
       u'appPlatform'],
      dtype='object')


In [9]:
print len(train.columns)
print len(test.columns)

21
20


In [8]:
del test["instanceID"]

In [26]:
del train["conversionTime"]
print train.head()

   label  clickTime  creativeID   userID  positionID  connectionType  \
0      0     170000        3089  2798058         293               1   
1      0     170000        1259   463234        6161               1   
2      0     170000        4465  1857485        7434               4   
3      0     170000        1004  2038823         977               1   
4      0     170000        1887  2015141        3688               1   

   telecomsOperator  
0                 1  
1                 2  
2                 1  
3                 1  
4                 1  


In [35]:
del test["label"]
del test["instanceID"]
print test.head()

   clickTime  creativeID   userID  positionID  connectionType  \
0     310000        3745  1164848        3451               1   
1     310000        2284  2127247        1613               1   
2     310000        1456  2769125        5510               2   
3     310000        4565     9762        4113               2   
4     310000          49  2513636        3615               1   

   telecomsOperator  
0                 3  
1                 3  
2                 1  
3                 3  
4                 3  


In [10]:
# split the train data
train_xy,val=train_test_split(train,test_size=0.3,random_state=1)
y=train_xy.label
X=train_xy.drop(["label"],axis=1)
val_y=val.label
val_X=val.drop(["label"],axis=1)

# create xgb matrix
xgb_val=xgb.DMatrix(val_X,label=val_y)
xgb_train=xgb.DMatrix(X,label=y)

In [11]:
xgb_test=xgb.DMatrix(test)

In [16]:
print len(train_xy)
print len(val)

2624669
1124859


In [21]:
# find the scale param
pos_sum=train["label"].sum()

In [23]:
neg_sum=len(train[train["label"]==0])
print neg_sum

3656266


In [24]:
print len(train)

3749528


In [22]:
print pos_sum

93262


In [27]:
scale_pos_weight=float(pos_sum)/neg_sum
print scale_pos_weight

0.0255074439332


In [12]:
params={
    "booster":"gbtree",
    "objective":"binary:logistic",
    "gamma":0.1,
    "max_depth":10,
    "lambda":2,
    "subsample":0.8, #it has been divided before
    "colsample_bytree":0.7,# may be need for tree
    "min_child_weight":1,
    "silent":0,
    "eta":0.05,
    "seed":1000,
    "scale_pos_weight":1 # best set for n/r
}
plst=list(params.items())
num_rounds=5000
watchlist=[(xgb_train,"train"),(xgb_val,"val")]

model = xgb.train(plst, xgb_train, num_rounds, watchlist,early_stopping_rounds=100)
model.save_model('dataset/pre/xgb1.model')
print "best best_ntree_limit",model.best_ntree_limit

[0]	train-error:0.02489	val-error:0.024842


Multiple eval metrics have been passed: 'val-error' will be used for early stopping.



Will train until val-error hasn't improved in 100 rounds.


[1]	train-error:0.024901	val-error:0.024792


[2]	train-error:0.024906	val-error:0.024776


...

[104]	train-error:0.0249	val-error:0.024775


[105]	train-error:0.0249	val-error:0.024775


[106]	train-error:0.0249	val-error:0.024775


Stopping. Best iteration:
[6]	train-error:0.024909	val-error:0.024768



best best_ntree_limit 7


In [13]:
preds=model.predict(xgb_test,ntree_limit=model.best_ntree_limit)
np.savetxt("dataset/pre/xgb_submission3.csv",np.c_[range(1,len(test)+1),preds],delimiter=',',header='instanceID,prob',comments='',fmt='%.4f')
cost_time=time.time()-start_time
print "xgboost success!",'\n',"cost time:",cost_time,"(s)......"

xgboost success! 
cost time: 2895.35550809 (s)......


In [18]:
a=0.233333
print '%.2f' a

SyntaxError: invalid syntax (<ipython-input-18-795bcb5747fe>, line 2)

In [40]:
sub=pd.read_csv("xgb_submission.csv")
print sub["Prob"].unique()

[0]


In [17]:
test=pd.read_csv("dataset/pre/test.csv")
sub_test=test[[test.columns[0],test.columns[1]]]
sub_test["prob"]=0
print sub_test.head()

   instanceID  label  prob
0           1     -1     0
1           2     -1     0
2           3     -1     0
3           4     -1     0
4           5     -1     0


/Users/suncicie/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [18]:
result=sub_test[["instanceID","prob"]]
print result.head()
result.to_csv("dataset/pre/result01.csv")

   instanceID  prob
0           1     0
1           2     0
2           3     0
3           4     0
4           5     0
